<a href="https://colab.research.google.com/github/pinilDissanayaka/Skin-Diseases-Image-Classification/blob/main/Skin_Diseases_Using_Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [3]:
! kaggle datasets download haroonalam16/20-skin-diseases-dataset

20-skin-diseases-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip /content/20-skin-diseases-dataset.zip

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from glob import glob
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.metrics import F1Score
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings(action = 'ignore')

In [10]:
IMG_SIZE = [224, 224, 3]
basedir = '/content/Dataset'

In [19]:
traindir = basedir + '/' + 'train'
testdir = basedir + '/' + 'test'

In [25]:
print(f"Traing set has {len(os.listdir(traindir))} folders")
print(f"Testing set has {len(os.listdir(testdir))} folders")

outputsize = len(os.listdir(traindir))

Traing set has 19 folders
Testing set has 19 folders


In [30]:
resnet = ResNet50V2(include_top= False, weights = 'imagenet', input_shape = IMG_SIZE)

In [31]:
for layer in resnet.layers:
  layer.trainable = False

In [32]:
x = Flatten()(resnet.output)

x = Dense(units = 128, activation = 'relu')(x)
x = Dense(units = outputsize, activation = 'softmax')(x)

In [33]:
model = Model(inputs = [resnet.input], outputs = [x])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 114, 114, 64)         0         ['conv1_conv[0][0]']          
                                                                                              

In [ ]:
metrics = [F1Score()]

In [35]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = metrics)

In [36]:
earlystopping = EarlyStopping(monitor='val_loss', verbose=1, patience=1)